In [1]:
import pandas as pd
import numpy as np


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [108]:
df=pd.read_csv('missile_attacks_daily.csv')
df.head()

,time_start,time_end,model,launch_place,target,carrier,launched,destroyed,not_reach_goal,cross_border_belarus,back_russia,still_attacking,affected region,destroyed_details,launched_details,launch_place_details,source
0,2025-03-29 20:00,2025-03-30 09:00,Shahed-136/131,"Primorsko-Akhtarsk and Chauda, Crimea and Kurs...",Ukraine,NaN,111.0,65.0,35.0,NaN,NaN,NaN,"['Sumy oblast', 'Kharkiv oblast', 'Odesa oblas...","{'south': 5, 'north': NaN, 'east': NaN, 'cente...",NaN,NaN,kpszsu/posts/pfbid02nvBpjtAGTavMFEM3GxS2Nmoo27...
1,2025-03-29 20:00,2025-03-30 09:00,Ballistic Missile,Rostov oblast,Ukraine,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kpszsu/posts/pfbid02nvBpjtAGTavMFEM3GxS2Nmoo27...
2,2025-03-28 20:00,2025-03-29 08:30,Shahed-136/131,"Primorsko-Akhtarsk and Chauda, Crimea and Kurs...",Ukraine,NaN,172.0,94.0,69.0,NaN,NaN,NaN,"['Sumy oblast', 'Kharkiv oblast', 'Dnipropetro...","{'south': 1, 'north': NaN, 'east': NaN, 'cente...",NaN,NaN,kpszsu/posts/pfbid02DBuVh2KAtowVp454mWL8cjCnSq...
3,2025-03-27 18:00,2025-03-28 09:00,Shahed-136/131,"Primorsko-Akhtarsk and Chauda, Crimea and Kurs...",Ukraine,NaN,163.0,89.0,51.0,NaN,NaN,NaN,"['Sumy oblast', 'Kharkiv oblast', 'Dnipropetro...","{'south': 12, 'west': 2, 'north': NaN, 'center...",NaN,NaN,kpszsu/posts/pfbid023uwKRv8TaxB43CW3HzXyhVQzYh...
4,2025-03-28,2025-03-28,Unknown UAV,NaN,south,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PvKPivden/posts/pfbid02WBxs2upH9CqxAMwFpoASGTm...


In [7]:
df.target.value_counts()[:10]

Ukraine                                    842
south                                      431
Odesa oblast                               159
Kherson oblast                             150
Mykolaiv oblast                             96
east                                        91
Kharkiv oblast                              68
Dnipropetrovsk oblast                       45
Dnipropetrovsk oblast, Dnipro Raion         33
Dnipropetrovsk oblast, Kryvyi Rih Raion     26
Name: target, dtype: int64

In [11]:
len(df[df['target'].str.contains('Kyiv', case=False, na=False)])

42

In [95]:
df.model.value_counts()[:20]

Shahed-136/131        598
Orlan-10              181
Iskander-M            166
Supercam              165
ZALA                  164
X-59                  151
Lancet                124
X-59/X-69              85
X-101/X-555            74
C-300                  68
Unknown UAV            55
Kalibr                 53
Merlin-VR              42
X-31P                  40
Reconnaissance UAV     39
X-47 Kinzhal           34
Iskander-M/KN-23       32
X-22                   32
Iskander-K             31
Unknown Missile        24
Name: model, dtype: int64

In [119]:
# number of aerial attacks per annum
df['year']=pd.to_datetime(df['time_start']).dt.year
attacks_per_annum=pd.DataFrame(df.groupby('year').size()).reset_index()
attacks_per_annum.columns=['year','quantity']
attacks_per_annum

,year,quantity
0,2022,103
1,2023,528
2,2024,1349
3,2025,306


In [109]:
# means of attack
df['model_group'] = df['model']
df['model_group'] = df['model_group'].replace({
    'Orlan-10': 'Other drones',
    'X-59': 'X-59/X-69',
    'Lancet': 'Other drones',
    'Unknown UAV': 'Other drones',
    'Iskander-M/KN-23': 'Iskander',
    'Iskander-M': 'Iskander',
    'Iskander-K': 'Iskander',
    'Merlin-VR': 'Other drones',
    'Supercam': 'Other drones',
    'ZALA': 'Other drones',
    'Reconnaissance UAV': 'Other drones',
    'C-300': 'C-300/C-400',
    'X-101/X-555 and Kalibr': 'X-101/X-555',
    'X-101/X-555 and Kalibr and Iskander-K': 'X-101/X-555',
    'X-101/X-555 and Kalibr and X-59/X-69': 'X-101/X-555',
    'X-101/X-555 and X-22 and Kalibr': 'X-101/X-555',
    'Iskander-M and Iskander-K': 'Iskander',
    'C-300/C-400 and Iskander-M': 'C-300/C-400',
    'C-400 and Iskander-M': 'C-300/C-400',
    'X-101/X-555 and Kalibr and X-59/X-69': 'X-101/X-555',
    'X-22': 'X-22/X-32',
    'X-22 and X-32': 'X-22/X-32',
    'X-101/X-555 and Kalibr and Iskander-M/KN-23': 'Kalibr'
})

# Replace NaN (unmapped values) with 'Other models'
#df['model_group'] = df['model_group'].fillna('Other models')


In [110]:
top_models = df.model_group.value_counts()

top_models

Other drones                                   770
Shahed-136/131                                 598
X-59/X-69                                      236
Iskander                                       231
C-300/C-400                                     92
X-101/X-555                                     85
Kalibr                                          54
X-31P                                           40
X-22/X-32                                       37
X-47 Kinzhal                                    34
Unknown Missile                                 24
Молнія                                          16
P-800 Oniks                                      7
X-31                                             6
Orion                                            6
X-35                                             5
KAB                                              4
C-400                                            4
Mohajer-6                                        3
KN-23                          

In [111]:
models_launched=df.groupby('model_group')['launched'].sum().sort_values(ascending=False)[:9].reset_index()
models_launched.columns=['model_name', 'quantity_launched']
total_attacks=np.sum(df.groupby('model_group')['launched'].sum())
other_models=total_attacks-models_launched['quantity_launched'].sum()
models_launched.loc[len(models_launched)] = ['Other models', other_models]
models_launched

,model_name,quantity_launched
0,Shahed-136/131,25084.0
1,X-101/X-555,2295.0
2,Other drones,1345.0
3,X-59/X-69,477.0
4,Iskander,467.0
5,C-300/C-400,459.0
6,Kalibr,361.0
7,X-22/X-32,177.0
8,X-47 Kinzhal,122.0
9,Other models,361.0


In [62]:
df.target.value_counts()[:30]

Ukraine                                     842
south                                       431
Odesa oblast                                159
Kherson oblast                              150
Mykolaiv oblast                              96
east                                         91
Kharkiv oblast                               68
Dnipropetrovsk oblast                        45
Dnipropetrovsk oblast, Dnipro Raion          33
Dnipropetrovsk oblast, Kryvyi Rih Raion      26
Zaporizhzhia oblast                          23
Kyiv oblast                                  21
Donetsk oblast                               20
Sumy oblast                                  16
Odesa oblast and Mykolaiv oblast             16
Poltava oblast                               13
Kryvyi Rih                                   11
Dnipro                                       10
Dnipropetrovsk oblast, Synelnykove Raion     10
Mykolaiv oblast and Kherson oblast           10
north                                   

In [58]:
df.target.isna().mean()

0.012248468941382326

In [78]:
# Air defence effectiveness
objects_destroyed=df.groupby('year')['destroyed'].sum()
objects_launched=df.groupby('year')['launched'].sum()
objects_summary = pd.concat([objects_launched, objects_destroyed], axis=1).reset_index()
objects_summary['effectiveness']=np.round(objects_summary['destroyed']/objects_summary['launched'],2)
objects_summary

,year,launched,destroyed,effectiveness
0,2022,1157.0,909.0,0.79
1,2023,4815.0,3846.0,0.80
2,2024,14112.0,9554.0,0.68
3,2025,11064.0,6507.0,0.59


In [77]:
objects_summary.iloc[1,2]/objects_summary.iloc[1,1]

0.7987538940809968

In [81]:
df['target'].isna().mean()

0.012248468941382326

In [84]:
#Target of attacks
cities=['Kyiv', 'Odesa', 'Dnipro', 'Kharkiv', 'Lviv']
compass=['north', 'south', 'east','east', 'west']
targets=[]
for city, direction in zip(cities, compass):
    pattern = fr'{city}|Ukraine|{direction}'
    count = df['target'].str.contains(pattern, case=False, na=False).sum()
    targets.append(count)
    
targets_of_attack = pd.DataFrame({
    'city': cities,
    'attack_count': targets
})
targets_of_attack


,city,attack_count
0,Kyiv,899
1,Odesa,1487
2,Dnipro,1103
3,Kharkiv,1027
4,Lviv,848


In [88]:
################Prep
# Cost of war
weapon_prices = {
    "Shahed-136/131": 0.05,
    "X-101/X-555": 13.0,
    "Other drones": 0.2,
    "X-59/X-69": 0.5,
    "Iskander": 3.0,
    "C-300/C-400": 1.2,
    "Kalibr": 6.5,
    "X-22/X-32": 1.0,
    "X-47 Kinzhal": 10.0,
    "Other models":0.05
}
df['weapon_price'] = df['model_group'].map(weapon_prices)
df['cost_of_attack'] = df['weapon_price'] * df['launched']

In [91]:
cost_of_war=df.groupby('year')['cost_of_attack'].sum().reset_index()
cost_of_war

,year,cost_of_attack
0,2022,7481.75
1,2023,11786.45
2,2024,15972.35
3,2025,2051.45


In [113]:
cities = ['Kyiv', 'Odesa', 'Dnipro', 'Kharkiv', 'Lviv']
compass = ['north', 'south', 'east', 'east', 'west']

# Create an empty list to store target counts
targets = []

# Loop through the cities and compass directions to count the attacks per year
for city, direction in zip(cities, compass):
    pattern = fr'{city}|Ukraine|{direction}'
    # Group by year and city, and count occurrences of the pattern
    city_targets = df[df['target'].str.contains(pattern, case=False, na=False)]
    yearly_counts = city_targets.groupby('year').size().reset_index(name='attack_count')
    yearly_counts['city'] = city
    targets.append(yearly_counts)

# Combine all targets for each city into one DataFrame
targets_of_attack = pd.concat(targets)
targets_of_attack

,year,attack_count,city
0,2022,58,Kyiv
1,2023,253,Kyiv
2,2024,462,Kyiv
3,2025,126,Kyiv
0,2022,73,Odesa
1,2023,296,Odesa
2,2024,837,Odesa
3,2025,281,Odesa
0,2022,60,Dnipro
1,2023,303,Dnipro


In [122]:
weapon_prices = {
    "Shahed-136/131": 0.05,
    "X-101/X-555": 13.0,
    "Other drones": 0.2,
    "X-59/X-69": 0.5,
    "Iskander": 3.0,
    "C-300/C-400": 1.2,
    "Kalibr": 6.5,
    "X-22/X-32": 1.0,
    "X-47 Kinzhal": 10.0,
    "Other models":0.05}
df['weapon_price'] = df['model_group'].map(weapon_prices)
df['cost_of_attack'] = df['weapon_price'] * df['launched']

In [123]:
cost_of_war=pd.DataFrame(df.groupby('year')['cost_of_attack'].sum()).reset_index()
cost_of_war.columns=['year','cost_of_attack']
cost_of_war

,year,cost_of_attack
0,2022,7481.75
1,2023,11786.45
2,2024,15972.35
3,2025,2051.45
